In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the URL
url = "https://www.popvortex.com/music/charts/top-100-songs.php"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all p tags with class 'title-artist'
title_artist_tags = soup.find_all('p', class_='title-artist')

# Extract titles and artists from the tags
titles = []
artists = []
for tag in title_artist_tags:
    title = tag.find('cite', class_='title').text.strip()
    artist = tag.find('em', class_='artist').text.strip()
    titles.append(title)
    artists.append(artist)

# Create a DataFrame
data = {'Title': titles, 'Artist': artists}
itunes_df = pd.DataFrame(data)

# df.to_csv('music_chart_data.csv', index=False)

# Display the DataFrame
df.head()


,Title,Artist
0,Lovin On Me,Jack Harlow
1,Lose Control,Teddy Swims
2,Beautiful Things,Benson Boone
3,Cruel Summer,Taylor Swift
4,Snooze,SZA


# Lab 2 expand data



In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the URL
url = "https://www.billboard.com/charts/hot-100/"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all list items with class 'o-chart-results-list__item'
song_items = soup.find_all('li', class_='o-chart-results-list__item')

# Extract titles and artists from the list items
data = {'Title': [], 'Artist': []}
for item in song_items:
    title_elem = item.find('h3', class_='c-title')
    artist_elem = item.find('span', class_='c-label')
    if title_elem and artist_elem:
        title = title_elem.text.strip()
        artist = artist_elem.text.strip()
        data['Title'].append(title)
        data['Artist'].append(artist)

# Create a DataFrame
billboard_df = pd.DataFrame(data)

# Display the DataFrame
billboard_df.tail(20)


,Title,Artist
80,Standing Next To You,Jung Kook
81,Perro Negro,Bad Bunny & Feid
82,Mmhmm,BigXthaPlug
83,Oklahoma Smokeshow,Zach Bryan
84,My Eyes,Travis Scott
85,Coal,Dylan Gossett
86,A Cold Sunday,Lil Yachty
87,All Of Me,21 Savage
88,N.H.I.E.,21 Savage & Doja Cat
89,Should've Wore A Bonnet,21 Savage & Brent Faiyaz


In [16]:
# Concatenate both DataFrames vertically
combined_df = pd.concat([itunes_df, billboard_df], ignore_index=True)

# Display the combined DataFrame
print(combined_df)


                       Title                       Artist
0             TEXAS HOLD 'EM                      Beyoncé
1               Lose Control                  Teddy Swims
2           Beautiful Things                 Benson Boone
3                    Flowers                  Miley Cyrus
4    Turn the Lights Back On                   Billy Joel
..                       ...                          ...
195                 Worth It         Offset & Don Toliver
196        Northern Attitude       Noah Kahan With Hozier
197          Scared To Start              Michael Marcagi
198               First Love  Oscar Ortiz X Edgardo Nunez
199            Where It Ends             Bailey Zimmerman

[200 rows x 2 columns]


In [17]:
# Save the merged DataFrame to a CSV file
combined_df.to_csv('combined_df.csv', index=False)